# AOC 2023

Welcome to the Advent of Code 2023!

## Basic configuration



In [ ]:
! pip install -U advent-of-code-data numpy pandas networkx matplotlib scipy

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

In [ ]:
from aocd.models import Puzzle
from collections import Counter, defaultdict
from itertools import chain, product
from numpy.typing import ArrayLike
from pprint import pprint
from queue import PriorityQueue
from scipy import sparse
from statistics import median
from typing import Callable, Dict, Iterator, Mapping, Tuple
import copy
import functools
import math
import operator
import networkx as nx
import numpy as np
import pandas as pd
import sys
import re

## Day 25
https://adventofcode.com/2023/day/25

In [ ]:
puzzle = Puzzle(year=2023, day=25)
input_data = """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr"""

### Part 1

In [ ]:
G = nx.Graph()

for line in puzzle.input_data.splitlines():
    src, dests = line.split(": ")
    for dest in dests.split(" "):
        G.add_edge(src, dest)

In [ ]:
for edge in nx.minimum_edge_cut(G):
    G.remove_edge(*edge)

In [ ]:
total = 1
for node_set in nx.connected_components(G):
    total *= len(node_set)
puzzle.answer_a = total

## Day 24
https://adventofcode.com/2023/day/24

In [ ]:
puzzle = Puzzle(year=2023, day=24)

In [ ]:
input_data = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3"""

In [ ]:
hailstones = []  # [pos, vel] each is (x, y, z)
for line in puzzle.input_data.splitlines():
    pos, vel = [list(map(int, x.split(", "))) for x in line.split(" @ ")]
    hailstones.append([pos, vel])

### Part 1

In [ ]:
min_test_area = 200000000000000
max_test_area = 400000000000000

In [ ]:
collisions = 0
debug = False

for i in range(len(hailstones)):
    for j in range(i+1, len(hailstones)):
        hi = hailstones[i]
        hj = hailstones[j]
        
        if debug:
            print(hi, hj)
        
        ratio = hi[1][1] / hi[1][0]
        delta = hj[1][1] - hj[1][0] * ratio
        
        if delta == 0:
            if debug:
                print("Colinears")
            continue
        
        dj_num = hi[0][1] - hj[0][1] + ratio * (hj[0][0] - hi[0][0])
        dj = dj_num / delta

        inter_x = hj[0][0] + hj[1][0] * dj
        inter_y = hj[0][1] + hj[1][1] * dj

        di = (inter_x - hi[0][0]) / hi[1][0]


        if min_test_area <= inter_x <= max_test_area and min_test_area <= inter_y <= max_test_area:
            if dj >= 0 and di >= 0:
                if debug:
                    print("success :", inter_x, inter_y, di, dj)
                collisions += 1
            elif debug:
                print("before start :", di, dj)
        elif debug:
            print("outside :", inter_x, inter_y, di, dj)

puzzle.answer_a = collisions

### Part 2

## Day 23
https://adventofcode.com/2023/day/23

In [ ]:
puzzle = Puzzle(year=2023, day=23)
input_data = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#"""

### Part 1

In [ ]:
grid = np.array([list(x) for x in puzzle.input_data.splitlines()])
nb_rows, nb_cols = grid.shape

In [ ]:
print("\n". join(["".join(x) for x in grid]))

In [ ]:
paths = [[(nb_rows-1, nb_cols-2), 1, None]]  # (cur_x, cur_y), length, (prev_x, prev_y)
final_paths = []
ALLOWED_SYMBOLS = {
    (1, 0): "<",
    (-1, 0): ">",
    (0, 1): "^",
    (0, -1): "v"
}

while paths:
    path = paths.pop(0)
    
    cursor = path[0]
    
    # Termination criteria
    if cursor == (0, 1):
        final_paths.append(path[1])
        continue
    
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if bool(dx) != bool(dy) and 0 <= cursor[0] + dy < nb_rows and 0 <= cursor[1] + dx < nb_cols:
                if grid[cursor[0] + dy, cursor[1] + dx] in [".", ALLOWED_SYMBOLS[(dx, dy)]] and (cursor[0] + dy, cursor[1] + dx) != path[2]:
                    
                    paths.append([(cursor[0] + dy, cursor[1] + dx), path[1] + 1, cursor])
                    
puzzle.answer_a = max(final_paths) - 1

### Part 2

In [ ]:
paths = [[(nb_rows-1, nb_cols-2), 1, None, []]]  # (cur_x, cur_y), length, (prev_x, prev_y), crossings
final_paths = []

while paths:
    path = paths.pop(0)
    
    cursor = path[0]
    
    # Termination criteria
    if cursor == (0, 1):
        final_paths.append(path[1])
        continue
    
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if bool(dx) != bool(dy) and 0 <= cursor[0] + dy < nb_rows and 0 <= cursor[1] + dx < nb_cols:
                if grid[cursor[0] + dy, cursor[1] + dx] != "#" and (cursor[0] + dy, cursor[1] + dx) != path[2] and (cursor[0] + dy, cursor[1] + dx) not in path[3]:
                    if grid[cursor[0] + dy, cursor[1] + dx] != ".":
                        new_crossings = copy.deepcopy(path[3]) + [cursor]
                    else:
                        new_crossings = path[3]
                    paths.append([(cursor[0] + dy, cursor[1] + dx), path[1] + 1, cursor, new_crossings])
                    
puzzle.answer_b = max(final_paths) - 1

## Day 22
https://adventofcode.com/2023/day/22

In [ ]:
puzzle = Puzzle(year=2023, day=22)
input_data = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""

### Part 1

In [ ]:
# SUPER HARD

### Part 2

## Day 21
https://adventofcode.com/2023/day/21

In [ ]:
puzzle = Puzzle(year=2023, day=21)
input_data = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

In [ ]:
ATTEMPTS = [
    [0, 1],
    [0, -1],
    [1, 0],
    [-1, 0]
]

In [ ]:
grid = np.array([list(x) for x in input_data.splitlines()])
nb_rows, nb_cols = grid.shape
row_start, col_start = np.where(grid=="S")

reachables = {(row_start[0], col_start[0])}

### Part 1

In [ ]:
for step in range(64):
    next_reachables = set()
    for elem in reachables:
        for possibility in ATTEMPTS:
            next_target = (elem[0] + possibility[0], elem[1] + possibility[1])
            if 0 <= next_target[0] < nb_rows and 0 <= next_target[1] < nb_cols and grid[next_target] != "#":
                next_reachables.add(next_target)
    reachables = next_reachables

In [ ]:
puzzle.answer_a = len(reachables)

### Part 2

In [ ]:
# TODO : Dynamic programming ?

for step in range(400):
    next_reachables = set()
    for elem in reachables:
        for possibility in ATTEMPTS:
            next_target = (elem[0] + possibility[0], elem[1] + possibility[1])
            next_target_mod = (next_target[0] % nb_rows, next_target[1] % nb_cols)
            if grid[next_target_mod] != "#":
                next_reachables.add(next_target)
    reachables = next_reachables
len(reachables)

## Day 20
https://adventofcode.com/2023/day/20

In [ ]:
puzzle = Puzzle(year=2023, day=20)
input_data = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

input_data = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [ ]:
class FlipFlop:
    # prefix %
    
    def __init__(self, name, dests):
        self.name = name
        self.dests = dests
        self.module = False  # off
        
    def handle(self, signal, source=None):
        """Low pulse is stored as a False bool, high_pulse is True"""
        if signal is False:
            self.module = not self.module
            return [self.module, self.dests]  # a signal : high if on, low if off
        
        return [None, []]

class Conjonction:
    # prefix &
    
    def __init__(self, name, dests):
        self.name = name
        self.inputs = defaultdict(bool)
        self.dests = dests

    def add_input(self, name):
        self.inputs[name] = False
        
    def handle(self, signal, source):        
        self.inputs[source] = signal
        
        return [not all(self.inputs.values()), self.dests]
        
class Broadcaster:
    def __init__(self, dests):
        self.dests = dests
    
    def handle(self, signal=None, source=None):
        return [False, self.dests]
    
class Inactive:
    def __init__(self, *args):
        self.received_signals = {False: 0, True: 0}
        
    def reset_counters(self):
        self.received_signals = {False: 0, True: 0}
        
    def handle(self, signal, source=None):
        self.received_signals[signal] += 1
        
        return [None, []]

In [ ]:
def initiate_modules():
    modules = {}

    for line in puzzle.input_data.splitlines():

        source, dests = line.split(" -> ")
        dests = dests.split(", ")
        if source == "broadcaster":
            modules[source] = Broadcaster(dests)

        elif source.startswith("%"):
            modules[source[1:]] = FlipFlop(source[1:], dests)

        elif source.startswith("&"):
            modules[source[1:]] = Conjonction(source[1:], dests)

    all_modules = copy.deepcopy(modules)

    for name, module in modules.items():
        for d in module.dests:
            if isinstance(modules.get(d), Conjonction):
                all_modules[d].add_input(name)

            if d not in all_modules:
                all_modules[d] = Inactive()

    return all_modules

In [ ]:
def press_button(modules):

    pulses = {False: 0, True: 0}  # pulse counter
    
    for m in modules.values():
        if isinstance(m, Inactive):
            m.reset_counters()
        
    storage = [("broadcaster", False, "button", 0)]  # dest, signal, source, timestamp

    while storage:
        next_pulse = storage.pop(0)
        name = next_pulse[0]
        timestamp = next_pulse[3]

        # print(next_pulse[2], "-", next_pulse[1], "->", name, timestamp)
        pulses[next_pulse[1]] += 1
        
        module = modules.get(name)
        
        if module is not None:
            pulse, dests = module.handle(next_pulse[1], next_pulse[2])
            for d in dests:
                storage.append((d, pulse, name, timestamp + 1))
            
    # print(pulses)
    return pulses, modules

### Part 1

In [ ]:
total = {False: 0, True: 0}

modules = initiate_modules()

for i in range(1000):
    pulses, modules = press_button(modules)
    
    for k, v in pulses.items():
        total[k] += v
        
puzzle.answer_a = total[False] * total[True]

### Part 2

In [ ]:
press = 0

modules = initiate_modules()

while True:
    press += 1
    
    _, modules = press_button(modules)
    # print(modules["rx"].received_signals)
    if modules["rx"].received_signals[False] == 1:
        puzzle.answer_b = press
        break

## Day 19

https://adventofcode.com/2023/day/19

In [ ]:
puzzle = Puzzle(year=2023, day=19)

input_data = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

### Part 1

In [ ]:
workflows_str, parts_str = input_data.split("\n\n")
workflows_str = workflows_str.splitlines()
parts_str = parts_str.splitlines()

In [ ]:
parts = []
for part in parts_str:
    elems = part[1:-1].split(",")
    new_part = {}
    for e in elems:
        key, val = e.split("=")
        new_part[key] = val
    parts.append(new_part)

In [ ]:
workflows = defaultdict(list)
for workflow in workflows_str:
    name, content = workflow.split("{")
    rules = content[:-1].split(",")
    workflows[name] = rules

In [ ]:
total = 0

for part in parts:
    workflow = "in"
    while workflow not in ["A", "R"]:
        rules = copy.deepcopy(workflows[workflow])
        # print(workflow)
        while rules:
            rule = rules.pop(0)
            
            if ":" in rule:
                cond, success = rule.split(":")
                for k, v in part.items():
                    cond = cond.replace(k, v)

                if eval(cond):
                    workflow = success
                    break
            else:
                workflow = rule
                break  # should be useless
    if workflow == "A":
        total += sum(map(int, part.values()))
        
puzzle.answer_a = total

### Part 2

In [ ]:
workflows

## Day 18
https://adventofcode.com/2023/day/18

In [ ]:
puzzle = Puzzle(year=2023, day=18)
input_data = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

### Part 1

In [ ]:
digged = defaultdict(lambda : defaultdict(bool))
digged[0][0] = True
pos = [0, 0]  # row_id, col_id

min_rows, max_rows, min_cols, max_cols = 0, 0, 0, 0

for line in puzzle.input_data.splitlines():
    direction, length = re.match("(\w) (\d+)", line).groups()
    for i in range(int(length)):
        pos[0] += 1 if direction == "D" else (-1 if direction == "U" else 0)
        pos[1] += 1 if direction == "R" else (-1 if direction == "L" else 0)

        digged[pos[0]][pos[1]] = True

        if pos[0] < min_rows:
            min_rows = pos[0]
        elif pos[0] > max_rows:
            max_rows = pos[0]
            
        if pos[1] < min_cols:
            min_cols = pos[1]
        elif pos[1] > max_cols:
            max_cols = pos[1]

# print(min_rows, max_rows, min_cols, max_cols)

grid = np.zeros((max_rows - min_rows + 3, max_cols - min_cols + 3), dtype=bool)  # gonna take an extra row and col empty all around for later !
nb_rows, nb_cols = grid.shape
# print(nb_rows, nb_cols)

for row_id, v in digged.items():
    for col_id, check in v.items():
        grid[row_id - min_rows + 1, col_id - min_cols + 1] = check

In [ ]:
grid_str = grid.astype("str")
grid_str[grid==True] = "#"
grid_str[grid==False] = "."
# print("\n". join(["".join(x[:nb_cols//2]) for x in grid_str]))

In [ ]:
queue = [(0, 0)]  # guaranteed to be "." in grid_str

while queue:
    pos = queue.pop(0)
    grid_str[pos] = " "
    
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if bool(dx) != bool(dy):
                nx = pos[0] + dx
                ny = pos[1] + dy
                if 0 <= nx < nb_rows and 0 <= ny < nb_cols and grid_str[nx, ny] == ".":
                    new_elem = (pos[0] + dx, pos[1] + dy)
                    if new_elem not in queue:
                        queue.append(new_elem)

# print(grid_str)

In [ ]:
puzzle.answer_a = len(np.where(grid_str != " ")[0])

### Part 2

In [ ]:
DIRECTIONS = ["R", "D", "L", "U"]

digged = defaultdict(lambda : defaultdict(bool))
digged[0][0] = True
pos = [0, 0]  # row_id, col_id

min_rows, max_rows, min_cols, max_cols = 0, 0, 0, 0

for line in input_data.splitlines():
    length, direction = re.match("\w \d+ \(#(\w{5})(\d)\)", line).groups()
    length = int(length, 16)
    direction = DIRECTIONS[int(direction)]
    print(direction, length)
    
    for i in range(int(length)):
        pos[0] += 1 if direction == "D" else (-1 if direction == "U" else 0)
        pos[1] += 1 if direction == "R" else (-1 if direction == "L" else 0)

        digged[pos[0]][pos[1]] = True

        if pos[0] < min_rows:
            min_rows = pos[0]
        elif pos[0] > max_rows:
            max_rows = pos[0]
            
        if pos[1] < min_cols:
            min_cols = pos[1]
        elif pos[1] > max_cols:
            max_cols = pos[1]

print(min_rows, max_rows, min_cols, max_cols)

# Can't bruteforce

## Day 17
https://adventofcode.com/2023/day/17

In [ ]:
puzzle = Puzzle(year=2023, day=17)
input_data = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""

In [ ]:
grid = np.array([[int(y) for y in x] for x in input_data.splitlines()])
nb_rows, nb_cols = grid.shape

### Part 1

In [ ]:
# HARD

In [ ]:
grid

### Part 2

## Day 16
https://adventofcode.com/2023/day/16

In [ ]:
puzzle = Puzzle(year=2023, day=16)
input_data = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""

In [ ]:
INCREMENTS = {
    "D": [1, 0],
    "U": [-1, 0],
    "R": [0, 1],
    "L": [0, -1]
}

In [ ]:
grid = np.array([list(x) for x in puzzle.input_data.splitlines()])
nb_rows, nb_cols = grid.shape

In [ ]:
def energize(beams):
    """"""
    
    directions = defaultdict(lambda : defaultdict(str))  # add char when this direction is used : R, D, L, U

    while beams:
        beam = beams.pop(0)
        # print("beam:", beam)
        direction = beam[2]

        if direction in directions[beam[0]][beam[1]]:
            continue

        directions[beam[0]][beam[1]] += direction

        if grid[beam[0], beam[1]] == ".":
            new_beams = [[beam[0] + INCREMENTS[direction][0], beam[1] + INCREMENTS[direction][1], direction]]
        elif grid[beam[0], beam[1]] == "|":
            if direction in ["D", "U"]:
                new_beams = [[beam[0] + INCREMENTS[direction][0], beam[1] + INCREMENTS[direction][1], direction]]
            else:
                new_beams = [[beam[0] - 1, beam[1], "U"], [beam[0] + 1, beam[1], "D"]]
        elif grid[beam[0], beam[1]] == "-":
            if direction in ["L", "R"]:
                new_beams = [[beam[0] + INCREMENTS[direction][0], beam[1] + INCREMENTS[direction][1], direction]]
            else:
                new_beams = [[beam[0], beam[1] - 1, "L"], [beam[0], beam[1] + 1, "R"]]
        elif grid[beam[0], beam[1]] == "/":
            if direction == "L":
                new_beams = [[beam[0] + 1, beam[1], "D"]]
            elif direction == "D":
                new_beams = [[beam[0], beam[1] - 1, "L"]]
            elif direction == "R":
                new_beams = [[beam[0] - 1, beam[1], "U"]]
            elif direction == "U":
                new_beams = [[beam[0], beam[1] + 1, "R"]]
        elif grid[beam[0], beam[1]] == "\\":
            if direction == "L":
                new_beams = [[beam[0] - 1, beam[1], "U"]]
            elif direction == "D":
                new_beams = [[beam[0], beam[1] + 1, "R"]]
            elif direction == "R":
                new_beams = [[beam[0] + 1, beam[1], "D"]]
            elif direction == "U":
                new_beams = [[beam[0], beam[1] - 1, "L"]]

        # print("new:", new_beams)
        for b in new_beams:
            if 0 <= b[0] < nb_rows and 0 <= b[1] < nb_cols:
                # print("added", b)
                beams.append(b)

    return sum([len(list(filter(bool, v.values()))) for k, v in directions.items()])

### Part 1

In [ ]:
beams = [[0, 0, "R"]]  # row_id, col_id, direction
puzzle.answer_a = energize(beams)

### Part 2

In [ ]:
max_energy = 0

for i in range(nb_rows):
    energy = energize([[i, 0, "R"]])
    # print(i, 0, "R", energy)
    if energy > max_energy:
        max_energy = energy
        
    energy = energize([[i, nb_cols - 1, "L"]])
    # print(i, nb_cols - 1, "L", energy)
    if energy > max_energy:
        max_energy = energy
        
for j in range(nb_cols):
    energy = energize([[0, j, "D"]])
    # print(0, j, "D", energy)
    if energy > max_energy:
        max_energy = energy
        
    energy = energize([[nb_rows - 1, j, "U"]])
    # print(nb_rows - 1, j, "U", energy)
    if energy > max_energy:
        max_energy = energy

puzzle.answer_b = max_energy

## Day 15
https://adventofcode.com/2023/day/15

In [ ]:
puzzle = Puzzle(year=2023, day=15)
input_data = "rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7"

In [ ]:
def hash_algo(word:str) -> int:
    total = 0
    for char in word:
        total += ord(char)
        total *= 17
        total %= 256
    return total

### Part 1

In [ ]:
for row in puzzle.input_data.splitlines():
    scores = []
    for word in row.split(","):
        scores.append(hash_algo(word))
puzzle.answer_a = sum(scores)

### Part 2

In [ ]:
for row in puzzle.input_data.splitlines():
    boxes = defaultdict(list)
    storage = {}

    for word in row.split(","):
        label = word.split("=")[0]
        
        if "-" in word:
            label = label[:-1]
            if label in storage:
                box_id = storage[label]
                del storage[label]
                
                for i, w in enumerate(boxes[box_id]):
                    if w.startswith(label):
                        del boxes[box_id][i]
                        break
                
        elif "=" in word:
            if label in storage:
                box_id = storage[label]
                
                for i, w in enumerate(boxes[box_id]):
                    if w.startswith(label):
                        boxes[box_id][i] = word
                        break
            else:  
                box_id = hash_algo(label)
                storage[label] = box_id
                boxes[box_id].append(word)


In [ ]:
total = 0
for box_id, box in boxes.items():
    for i, item in enumerate(box):
        focal = int(item.split("=")[1])
        total += (box_id+1) * (i+1) * focal
puzzle.answer_b = total

## Day 14
https://adventofcode.com/2023/day/14

In [ ]:
puzzle = Puzzle(year=2023, day=14)
input_data = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [ ]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [ ]:
def score(grid):
    nb_rows, nb_cols = grid.shape
    total = 0
    
    for x in range(nb_rows):
        total += Counter(grid[x, :]).get("O", 0) * (nb_rows - x)
    return total


### Part 1

In [ ]:
grid = np.array([list(x) for x in puzzle.input_data.splitlines()])
nb_rows, nb_cols = grid.shape

for x in range(nb_cols):
    grid[:, x] = list(tilt(grid[:, x].tolist(), reverse=True))

puzzle.answer_a = score(grid)

### Part 2

In [ ]:
def tilt(bucket: list, reverse: bool)-> str:
    """Tilt in the direction of last element unless reversed"""
    sorted_groups = []

    for group in "".join(bucket).split("#"):
        sorted_groups.append("".join(sorted(group, reverse=reverse)))
    
    return "#".join(sorted_groups)

In [ ]:
def cycle(grid):
    nb_rows, nb_cols = grid.shape

    sizes = [nb_cols, nb_rows] * 2
    for direction in range(4):  # N, W, S, E
        for x in range(sizes[direction]):
            if direction % 2:
                grid[x, :] = list(tilt(grid[x, :].tolist(), direction < 2))
            else:
                 grid[:, x] = list(tilt(grid[:, x].tolist(), direction < 2))
    return grid

In [ ]:
grid = np.array([list(x) for x in puzzle.input_data.splitlines()])

scores = []
for i in range(200):
    cycle(grid)
    scores.append(score(grid))

In [ ]:
# Toy example
period = scores[2:2+7]

period[(1000000000 - 2) % 7 - 1]

In [ ]:
period = scores[95:95+78]

puzzle.answer_b = period[(1000000000 - 95) % 78 - 1]

## Day 13

_TBD_

https://adventofcode.com/2023/day/13

In [ ]:
puzzle = Puzzle(year=2023, day=13)
input_data = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

### Part 1

In [ ]:
verticals, horizontals = [], []

for game in puzzle.input_data.split("\n\n"):
    game_np = np.array([list(x) for x in game.splitlines()])
    
    nb_rows, nb_cols = game_np.shape
    
    for s in range(nb_cols-1): # s + 0.5 being the symmetrical line candidate
        works_for_all = True
        for x in range(nb_cols):
            if s-x < 0 or s+1+x >= nb_cols:
                break
            if any(game_np[:, s-x] != game_np[:, s+1+x]):
                works_for_all = False
                break
        if works_for_all:
            verticals.append(s+1)
            break
    
    for s in range(nb_rows-1): # s + 0.5 being the symmetrical line candidate
        works_for_all = True
        for x in range(nb_rows):
            if s-x < 0 or s+1+x >= nb_rows:
                break
            if any(game_np[s-x, :] != game_np[s+1+x, :]):
                works_for_all = False
                break
        if works_for_all:
            horizontals.append(s+1)
            break
            
puzzle.answer_a = sum(horizontals) * 100 + sum(verticals)

### Part 2

In [ ]:
verticals, horizontals = [], []

for game in puzzle.input_data.split("\n\n"):
    game_np = np.array([list(x) for x in game.splitlines()])
    
    nb_rows, nb_cols = game_np.shape
    
    for s in range(nb_cols-1): # s + 0.5 being the symmetrical line candidate
        nb_smudges = 0
        for x in range(nb_cols):
            if s-x < 0 or s+1+x >= nb_cols or nb_smudges > 1:
                break
            nb_smudges += sum(game_np[:, s-x] != game_np[:, s+1+x])

        if nb_smudges == 1:
            verticals.append(s+1)
            break
    
    for s in range(nb_rows-1): # s + 0.5 being the symmetrical line candidate
        nb_smudges = 0
        for x in range(nb_rows):
            if s-x < 0 or s+1+x >= nb_rows or nb_smudges > 1:
                break
                
            nb_smudges += sum(game_np[s-x, :] != game_np[s+1+x, :])
                
        if nb_smudges == 1:
            horizontals.append(s+1)
            break
            
puzzle.answer_b = sum(horizontals) * 100 + sum(verticals)

## Day 12

_TBD_

https://adventofcode.com/2023/day/12

In [ ]:
puzzle = Puzzle(year=2023, day=12)
input_data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

### Part 1

In [ ]:
def nb_solutions(secret_chars: str, data:list) -> int:
    """Count how many posibilities to match data into the secret string"""

    nb_q = Counter(secret_chars)["?"]
    
    nb_sols = 0
    
    for i in range(2**nb_q):
        possibility = ("0" * nb_q + bin(i)[2:])[-nb_q:]
        possibility = possibility.replace("0", ".").replace("1", "#")
        
        splits = secret_chars.split("?")
        attempt = "".join([x[0] + x[1] for x in zip(splits, possibility)]) + splits[-1]
        
        if [len(x) for x in attempt.split(".") if x] == data:
            nb_sols += 1
    
    return nb_sols

In [ ]:
# BRUTE FORCE

total = 0
for row in input_data.splitlines():
    onsens, damages = row.split()
    # onsens = [x for x in onsens.strip(".").split(".") if x]
    total += nb_solutions(onsens, [int(x) for x in damages.split(",")])
puzzle.answer_a = total

In [ ]:
def f(onsens, damages):
    
    
    
    
    # ??? [], ###, [1, 1, 3]
    # ??? [1], ###, [1, 3]
    # ??? [1, 1] ###, [3]
    
    
    if len(rmn_onsens) == 0:
        return f(rmn_onsens[0], rmn_damages, [], [], nb_combi)
    
    # How to solve sub problem
    nb_q = Counter(onsens)["?"]
    
    nb_sols = 0
    
    for i in range(2**nb_q):
        possibility = ("0" * nb_q + bin(i)[2:])[-nb_q:]
        possibility = possibility.replace("0", ".").replace("1", "#")
        
        attempt = "".join([x[0] + x[1] for x in zip(onsens.split("?"), possibility)])
        
        if [len(x) for x in attempt.split(".") if x] == damages:
            print(attempt)
            nb_sols += 1
            
    return nb_sols * nb_combi

    total = 0
    for i in range(len(rmn_damages) + 1):
        # if trying to match too many elements, stop
        if sum(rmn_damages[:i]) + len(rmn_damages[:i]) - 1 > len(rmn_onsens[0]):
            break
            
        return f(rmn_onsens[0], rmn_damages[:i], rmn_onsens[1:], rmn_damages[i:], nb_combi * nb_sols) 

    
f("??", [1], ["??"], [1], 1)

In [ ]:
def f(onsens, damages, rmn_onsens, rmn_damages, nb_combi):
    if len(damages) == 0:
        if "#" not in onsens:
            2 ** Counter(onsens)["?"] # TODO
    
    if len(damages) - 1 + sum(damages) > len(onsens):
        print(onsens, damages, "no solution")
        return 0
    
    if len(damages) - 1 + sum(damages) == len(onsens):
        unique_sol = ".".join(["#" * d for d in damage])
        if all([s == o or o == "?" for s, o in zip(unique_sol, onsens)]):
            print("unique sol matches")
            if len(rmn_onsens) == 0:
                print("no more tests", len(rmn_damages), nb_combi)
                return nb_combi if len(rmn_damages) == 0 else 0
        
            for i in range(len(nb_damages) + 1):
                print("continue...")
                return f(rmn_onsens[0], rmn_damages[:i], rmn_onsens[1:], rmn_damages[i:], nb_combi)
            
    print(onsens, damages, rmn_onsens, rmn_damages, nb_combi)
    

### Part 2

## Day 11

_TBD_

https://adventofcode.com/2023/day/11

In [ ]:
puzzle = Puzzle(year=2023, day=11)
input_data = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

In [ ]:
board = np.array([list(x) for x in puzzle.input_data.splitlines()])
empty_rows, empty_cols = [], []
Y, X = board.shape

for i in range(X):
    if "#" not in board[:, i]:
        empty_cols.append(i)

for j in range(Y):
    if "#" not in board[j, :]:
        empty_rows.append(j)

pos = []
for i in range(X):
    for j in range(Y):
        if board[j, i] == "#":
            pos.append([i, j])

### Part 1

In [ ]:
def distance_with_malus(pos, coef=1):
    N = len(pos)
    total = 0
    for i, x in enumerate(pos):
        for j in range(i+1, N):
            y = pos[j]
            malus = len(set(empty_rows).intersection(set(range(min(x[1], y[1]), max(x[1], y[1]))))) + len(set(empty_cols).intersection(set(range(min(x[0], y[0]), max(x[0], y[0])))))
            dist = abs(x[0] - y[0]) + abs(x[1] - y[1]) + malus * coef
            total += dist
    return total

puzzle.answer_a = distance_with_malus(pos)

### Part 2

In [ ]:
puzzle.answer_b = distance_with_malus(pos, coef=999999)

## Day 10

_TBD_

https://adventofcode.com/2023/day/10

In [ ]:
puzzle = Puzzle(year=2023, day=10)
input_data = ""

### Part 1

In [ ]:
board = np.array([list(x) for x in puzzle.input_data.splitlines()])

In [ ]:
def get_next_move(prev_move, next_encounter):
    if next_encounter in ["|", "-"]:
        return prev_move
    elif next_encounter == "L":
        return (1, 0) if prev_move == (0, 1) else (0, -1)
    elif next_encounter == "J":
        return (-1, 0) if prev_move == (0, 1) else (0, -1)
    elif next_encounter == "7":
        return (-1, 0) if prev_move == (0, -1) else (0, 1)
    elif next_encounter == "F":
        return (1, 0) if prev_move == (0, -1) else (0, 1)
    print(next_encounter)

In [ ]:
ys, xs = np.where(board == "S")
xs, ys = xs[0], ys[0]

elements = []
encounter = (xs, ys, "S")

current_pos = (xs, ys)
next_move = (1, 0)  # noticed visually from the input

while encounter not in elements:
    # print("encountered:", encounter, "next is", next_move)
    elements.append(encounter)
    current_pos = (current_pos[0] + next_move[0], current_pos[1] + next_move[1])
    next_encounter = board[current_pos[1], current_pos[0]]
    next_move = get_next_move(next_move, next_encounter)
    encounter = (*current_pos, next_encounter)

len(elements) // 2

### Part 2

In [ ]:
X, Y = board.shape
board4 = np.zeros((2*X, 2*Y), dtype='<U1')

board[board != "S"] = "."
for elem in elements:
    board[elem[1], elem[0]] = elem[2]

for i in range(2*X):
    for j in range(2*Y):
        if i % 2 == 0 and j % 2 == 0:
            board4[j, i] = board[j//2, i//2]
        elif j % 2 == 0:
            board4[j, i] = "-" if board4[j, i-1] in ["F", "-", "L", "S"] else "."  # cheated : I know "S" is "-"
        elif i % 2 == 0:
            board4[j, i] = "|" if board4[j-1, i] in ["F", "|", "7"] else "."
        else:
            board4[j, i] = '.'

In [ ]:
print("\n". join(["".join(x) for x in board4]))

In [ ]:
queue = [(0, 0)]

while queue:
    change = queue.pop(0)
    board4[change[1], change[0]] = " "
    
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if bool(dx) != bool(dy):
                ny = change[1] + dy
                nx = change[0] + dx
                if 0 <= nx < 2*X and 0 <= ny < 2*Y and board4[ny, nx] == ".":
                    new_elem = (change[0] + dx, change[1] + dy)
                    if new_elem not in queue:
                        queue.append((change[0] + dx, change[1] + dy))

In [ ]:
np.count_nonzero(board4 == ".")

In [ ]:
total = 0
for i in range(X):
    for j in range(Y):
        total += 1 if board4[2*j, 2*i] == "." else 0
puzzle.answer_b = total

## Day 9

_TBD_

https://adventofcode.com/2023/day/9

In [ ]:
puzzle = Puzzle(year=2023, day=9)
input_data = """0 3 6 9 12 15
1 3 6 10 15 21
10 13 16 21 30 45"""

### Part 1

In [ ]:
row_val  = []
for row in puzzle.input_data.splitlines():
    full_storage = []
    values = [int(x) for x in row.split()]
    while any([x != 0 for x in values]):
        full_storage.append(values)
        values = [y-x for (x, y) in zip(values[:-1], values[1:])]
    
    add = 0
    for storage in full_storage[::-1]:
        add += storage[-1]
    row_val.append(add)

puzzle.answer_a = sum(row_val)

### Part 2

In [ ]:
row_val  = []
for row in puzzle.input_data.splitlines():
    full_storage = []
    values = [int(x) for x in row.split()]
    while any([x != 0 for x in values]):
        full_storage.append(values)
        values = [y-x for (x, y) in zip(values[:-1], values[1:])]
    
    add = 0
    for storage in full_storage[::-1]:
        add = storage[0] - add
    row_val.append(add)    

puzzle.answer_b = sum(row_val)

## Day 8

_dict, lcm_

https://adventofcode.com/2023/day/8

In [ ]:
puzzle = Puzzle(year=2023, day=8)
input_data = ""


In [ ]:
instructions = ""
adjacency = {}

for idx, line in enumerate(puzzle.input_data.splitlines()):
    if idx == 0:
        instructions = line
    elif idx >= 2:
        source, sinks = line.split(" = ")
        adjacency[source] = sinks[1:-1].split(", ")
instructions = [int(x) for x in instructions.replace("L", "0").replace("R", "1")]
N = len(instructions)

In [ ]:
# math.lcm for Python versions 3.9 and above
def lcm(denominators: list) -> int:
    """Compute LCM before it became native to python"""
    
    return functools.reduce(lambda a,b: a*b // math.gcd(a,b), denominators)

### Part 1

In [ ]:
def nb_steps(start: str, stop_rule) -> int:
    """Count the number of steps to reach the exit"""
    
    location = start
    counter = 0
    while stop_rule(location):
        location = adjacency[location][instructions[counter % N]]
        counter += 1
    
    return counter

In [ ]:
puzzle.answer_a = nb_steps("AAA", lambda location: location != "ZZZ")

### Part2

In [ ]:
scores = []

starting_points = [x for x in adjacency.keys() if x.endswith("A")]
for point in starting_points:
    scores.append(nb_steps(point, lambda location: not location.endswith("Z")))
    
puzzle.answer_b = lcm(scores)

## Day 7

_sorting_

https://adventofcode.com/2023/day/7

In [ ]:
puzzle = Puzzle(year=2023, day=7)
input_data = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

### Part 1

In [ ]:
hands = []

MAPPING = {"T": 10, "J": 11, "Q": 12, "K": 13, "A": 14}

for line in puzzle.input_data.splitlines():
    hand, bid = line.split()
    count = Counter(hand).most_common()
    hands.append([hand, count, int(bid)])
    
hands.sort(key=lambda x: ([e[1] for e in x[1]], [int(MAPPING.get(e, e)) for e in x[0]]))

puzzle.answer_a = sum([(i+1) * hand[2] for i, hand in enumerate(hands)])

### Part 2

In [ ]:
hands = []

MAPPING = {"T": 10, "J": 0, "Q": 12, "K": 13, "A": 14}

for line in puzzle.input_data.splitlines():
    hand, bid = line.split()
    count = Counter(hand)
    nb_J = count["J"]
    if nb_J < 5:
        del count["J"]
        sorted_count = count.most_common()
        sorted_count[0] = (sorted_count[0][0], sorted_count[0][1] + nb_J)
    else:
        sorted_count = count.most_common()
    hands.append([hand, sorted_count, int(bid)])
    
hands.sort(key=lambda x: ([e[1] for e in x[1]], [int(MAPPING.get(e, e)) for e in x[0]]))

puzzle.answer_b = sum([(i+1) * hand[2] for i, hand in enumerate(hands)])

## Day 6

_2nd order equation_

https://adventofcode.com/2023/day/6

In [ ]:
puzzle = Puzzle(year=2023, day=6)
input_data = """Time:      7  15   30
Distance:  9  40  200"""

In [ ]:
def nb_wins(races: list) -> int:
    """Compute the number of ways to win overall for a given input."""
    wins = 1

    for race in races.values():
        delta = math.sqrt(race[0] ** 2 - 4 * race[1])

        boundaries = [(race[0] - delta) / 2, (race[0] + delta) / 2]

        boundaries[0] = math.ceil(boundaries[0]) if boundaries[0] != math.ceil(boundaries[0]) else int(boundaries[0] + 1)
        boundaries[1] = math.floor(boundaries[1]) if boundaries[1] != math.floor(boundaries[1]) else int(boundaries[1] - 1)

        nb_wins = boundaries[1] - boundaries[0] + 1
        wins *= nb_wins
    
    return wins

### Part 1

In [ ]:
races = defaultdict(list)
for data in puzzle.input_data.splitlines():
    for i, val in enumerate(data.split()[1:]):
        races[i].append(int(val))

puzzle.answer_a = nb_wins(races)

### Part 2

In [ ]:
races = defaultdict(list)
for data in puzzle.input_data.splitlines():
    races[0].append(int("".join(data.split()[1:] + "00000000000000000000")))

puzzle.answer_b = nb_wins(races)

## Day 5

_TBD_

https://adventofcode.com/2023/day/5

In [ ]:
puzzle = Puzzle(year=2023, day=5)
input_data = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [ ]:
seeds = []
mappings = []
next_mapping = None

for line in puzzle.input_data.splitlines():
    if line.startswith("seeds"):
        seeds = [int(x) for x in line.lstrip("seeds: ").split()]
        
    elif "map" in line:
        next_mapping = []
        
    elif not line:
        if next_mapping:
            next_mapping = sorted(next_mapping, key=lambda m: m["source"])
            mappings.append(next_mapping)
    
    else:
        dest_start, source_start, size = [int(x) for x in line.split()]
        next_mapping.append({"source": source_start, "size": size, "dest": dest_start})
        
# no empty line at the end
next_mapping = sorted(next_mapping, key=lambda m: m["source"])
mappings.append(next_mapping)

In [ ]:
def process(value_ranges: list) -> int:
    """Do the full process for problem 5"""

    for mapping in mappings:
        # print()
        # print(value_ranges)
        # pprint(mapping)
        # print()

        next_values = []

        for v_range in value_ranges:
            first_value, range_size = v_range

            for range_def in mapping:

                if range_def["source"] <= first_value <= range_def["source"] + range_def["size"] - 1:
                    delta = first_value - range_def["source"]
                    next_first_val = range_def["dest"] + delta
                    next_size = min(range_def["size"] - delta, range_size)

                    next_values.append((next_first_val, next_size))
                    # print(f"Values {first_value}->{next_first_val}+{next_size} due to mapping {range_def}.")

                    range_size -= next_size
                    first_value += next_size

                if range_size == 0:
                    break

            else:
                next_values.append((first_value, range_size))
                # print(f"No mapping found for values {first_value}+{range_size}.")

        value_ranges = sorted(next_values, key=lambda x: x[0])

    return value_ranges[0][0]

### Part 1

In [ ]:
puzzle.answer_a = process(sorted([(s, 1) for s in seeds], key=lambda x: x[0]))

### Part 2

In [ ]:
puzzle.answer_b = process(sorted(zip(seeds[0::2], seeds[1::2]), key=lambda x: x[0]))

## Day 4

_TBD_

https://adventofcode.com/2023/day/4

In [ ]:
puzzle = Puzzle(year=2023, day=4)
input_data = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

### Part 1

In [ ]:
cards_info = {}

total_score = 0
for line in puzzle.input_data.splitlines():
    card_id_str, card_content = line.split(": ")
    card_id = int(card_id_str[5:])
    winning_nbs, my_nbs = card_content.split(" | ")
    winning_nbs = set([int(x.strip()) for x in winning_nbs.split()])
    my_nbs = set([int(x.strip()) for x in my_nbs.split()])
    cards_info[card_id] = len(winning_nbs.intersection(my_nbs))
    total_score += math.floor(2 ** (cards_info[card_id]-1))

puzzle.answer_a = total_score

### Part 2

In [ ]:
cards_info

copies = defaultdict(int)
for card_id, wins in cards_info.items():
    copies[card_id] += 1
    for x in range(wins):
        copies[card_id + x + 1] += copies[card_id]
        
puzzle.answer_b = sum(copies.values())

## Day 3

_text positioning, regexp_

https://adventofcode.com/2023/day/3

In [ ]:
puzzle = Puzzle(year=2023, day=3)

### Part 1

In [ ]:
storage = []

for idx, engine_row in enumerate(puzzle.input_data.splitlines()):
    symbols = [x for x in engine_row.split(".") if x]

    from_idx = -1
    for s in symbols:
        from_idx = engine_row.index(s, from_idx + 1)
        number = bool(re.fullmatch("\d+", s))
        if number:
            storage.append([s, idx, from_idx, True])
        elif len(s) == 1:
            storage.append([s, idx, from_idx, False])

        else:
            # sometimes numbers and symbols can be consecutive (by looking at the input, not more than number-symbol-number)
            number_symbol_number = re.fullmatch("(\d+)(\W)(\d+)", s)
            symbol_number = bool(re.search("\d+", s))
            number_symbol = bool(re.match("\d+", s))

            if number_symbol_number is not None:
                n1, symb, n2 = number_symbol_number.groups()
                storage.append([n1, idx, from_idx, True])
                storage.append([symb, idx, from_idx + len(n1), False])
                storage.append([n2, idx, from_idx + len(n1) + 1, True])
            elif number_symbol:
                storage.append([s[:-1], idx, from_idx, True])
                storage.append([s[-1], idx, from_idx + len(s) - 1, False])
            elif symbol_number:
                storage.append([s[0], idx, from_idx, False])
                storage.append([s[1:], idx, from_idx + 1, True])

            # there is no ELSE

            storage.append([s, idx, from_idx, number])
        from_idx += len(s)

numbers = [x for x in storage if x[3]]
symbols = [x for x in storage if not x[3]]

In [ ]:
total = 0

numbers_adjacency = []

for number in numbers:
    x = number[1]
    y_min = number[2]
    y_max = y_min + len(number[0]) - 1

    adj_symbols = []
    for symbol in symbols:
        if x - 1 <= symbol[1] <= x + 1 and y_min - 1 <= symbol[2] <= y_max + 1:
            adj_symbols.append(symbol)

        # they are ordered
        if symbol[1] > x + 1:
            break

    numbers_adjacency.append([number, adj_symbols])

puzzle.answer_a = sum([int(numb[0][0]) for numb in numbers_adjacency if numb[1]])

### Part 2

In [ ]:
gears = []

for symbol in symbols:
    if symbol[0] != "*":
        continue

    x = symbol[1]
    y = symbol[2]

    adj_numbers = []
    for number in numbers:
        if x - 1 <= number[1] <= x + 1 and any([y - 1 <= number[2] + y_numb <= y + 1 for y_numb in range(len(number[0]))]):
            adj_numbers.append(number)

        # they are ordered
        if number[1] > x + 1:
            break

    if len(adj_numbers) == 2:
        gears.append(int(adj_numbers[0][0]) * int(adj_numbers[1][0]))

puzzle.answer_b = sum(gears)

## Day 2

_defaultdict, min, max_

https://adventofcode.com/2023/day/2

In [ ]:
puzzle = Puzzle(year=2023, day=2)

In [ ]:
games = defaultdict(list)
for game in puzzle.input_data.splitlines():
    game_id_str, info = game.split(": ")
    game_id = int(game_id_str[5:])


    for look in info.split("; "):

        look_dict = {}
        for color_look in look.split(", "):
            value, color = color_look.split(" ")
            look_dict[color] = int(value)

        games[game_id].append(look_dict)

### Part 1

In [ ]:
hypothesis = {"red": 12, "green": 13, "blue": 14}

valid_games = []
for game_idx, game_content in games.items():
    valid_game = True
    for game_look in game_content:
        if not valid_game:
            break
        for color, value in game_look.items():
            if value > hypothesis[color]:
                valid_game = False
                break
    if valid_game:
        valid_games.append(game_idx)

puzzle.answer_a = sum(valid_games)

### Part 2

In [ ]:
total = 0

for game_idx, game_content in games.items():
    hypothesis = {"red": 0, "green": 0, "blue": 0}
    for game_look in game_content:
        for color, value in game_look.items():
            if value > hypothesis[color]:
                hypothesis[color] = value

    total += functools.reduce(lambda x, y: x*y, hypothesis.values(), 1)

puzzle.answer_b = total

## Day 1

_pandas, regexp_

https://adventofcode.com/2023/day/1

In [ ]:
puzzle = Puzzle(year=2023, day=1)



### Part 1

In [ ]:
calibration = pd.DataFrame(puzzle.input_data.splitlines())

In [ ]:
def get_calib_value(text: str) -> int:
    """Return the calibration: the integer formed by concatenating first and last digit."""

    digits = re.findall("\d", text)
    return int(f"{digits[0]}{digits[-1]}")

puzzle.answer_a = str(calibration.apply(lambda row: get_calib_value(row[0]), axis=1).sum())

### Part 2

In [ ]:
def numbers_text_to_int(text: str) -> str:
    """Transform text written in letters into digits, text can be overlapping."""

    # text before and after to be able to reuse characters (overlapping strings)
    replacements = [
        ("zero", "zero0zero"),
        ("one", "one1one"),
        ("two", "two2two"),
        ("three", "three3three"),
        ("four", "four4four"),
        ("five", "five5five"),
        ("six", "six6six"),
        ("seven", "seven7seven"),
        ("eight", "eight8eight"),
        ("nine", "nine9nine"),
    ]

    for rep_str, rep_int in replacements:
        text = text.replace(rep_str, rep_int)

    return text

puzzle.answer_b = str(calibration.apply(lambda row: get_calib_value(numbers_text_to_int(row[0])), axis=1).sum())